In [2]:
import numpy as np
from scipy import sparse
from string import Template


def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask


def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = rho * P - Q0.T.dot(tmp)

    return val, grad_P


def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho * P - Q.T.dot(tmp)

    grad_Q = rho * Q - tmp.dot(P.T)

    return val, grad_P, grad_Q


def total_objective_vectorized(PQvec, R, mask, rho):
    """
    Vectorisation de la fonction precedente de maniere a ne pas
    recoder la fonction gradient
    """

    # reconstruction de P et Q
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] / (n_items + n_users)
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))

    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])


### Question 1.1

L'option minidata déduit les données à une dimension 100 x 200 ( contre 943 x 1682 normalement)

In [3]:
R, mask = load_movielens("u.data")
R.shape

(943, 1682)

### Question 1.2

In [4]:
nbU = R.shape[0]
nbFilms = R.shape[1]
nbNotes = mask.sum()
print(Template("Il y a $u utilisateurs, $f films et $n notes.").substitute(u=nbU, f=nbFilms, n=nbNotes))

Il y a 943 utilisateurs, 1682 films et 100000 notes.


### Question 1.3
On nomme $ f $ la fonction objectif.

On calcul le gradient de $ f $ en calculant le gradient par rapport à P puis le gradient par rapport à Q :

$ ||P + h||^2 - ||P||^2  = 2tr(^{{\operatorname t}}\!P h) + o(h) = <2P, h> + o(h)$

$ ||1_K \circ (R - Q(P + h))||^2 - ||1_K \circ (R - QP)||^2 = -2tr( ^{{\operatorname t}}\!{(1_K \circ (R - QP))} 1_K \circ Qh) +o(h) = -2tr( ^{{\operatorname t}}\!{(1_K \circ (R - QP))} Qh) +o(h) = -2tr( ^{{\operatorname t}}\!{(^{{\operatorname t}}\!Q(1_K \circ (R - QP)))} h) + o(h) = <-2^{{\operatorname t}}\!Q(1_K \circ (R - Q P)), h> + o(h)$

On en déduit que le gradient de $ f $ par rapport à P est (en P, Q): 

$$ \rho P - ^{{\operatorname t}}\!Q(1_K \circ (R - Q P)) $$

Le calcul du gradient par rapport à Q est similaire.

La concaténation des 2 gradients donne : 

$$ {\overrightarrow {\nabla }}f (P, Q)=  \left(\rho P - ^{{\operatorname t}}\!Q(1_K \circ (R - Q P)),  \rho Q - (1_K \circ (R - Q P)) ^{{\operatorname t}}\!P \right)$$

(On ne se prononce pas sur la convexité de $f$)

### Question 2.1

On déduit facilement de la question précédente que le gradient de g en P est : $ \rho P - ^{{\operatorname t}}\!{Q^0}(1_K \circ (R - Q^0 P)) $

Par ailleurs, $ {\overrightarrow {\nabla }^2}g(P) = ^{{\operatorname t}}\!{Q^0}Q^0 + \rho I$ est définie positive donc g est convexe (et même strictement convexe).

### Question 2.2

Voir au dessus pour l'implémentation en python.

### Question 2.3

In [5]:
import math
from scipy.sparse.linalg import svds

nbC = 4
rho = 0.3

Q0, _, P0 = svds(R, k=nbC)
Q0TQ0 = Q0.T.dot(Q0)
gamma = 1. / (rho + math.sqrt(np.sum(Q0TQ0 ** 2)))
g = lambda P : objective(P, Q0, R, mask, rho)

In [6]:
def gradient(g, P0, gamma, epsilon, verbose = True, maxIter = 1000):
    P = P0.copy()
    _, grad = g(P)
    nbiter = 0
    while(math.sqrt(np.sum(grad ** 2)) > epsilon and nbiter < maxIter):
        P_new = P - grad * gamma
        P = P_new
        _, grad = g(P)
        nbiter += 1
    if(verbose):
        print("Réalisé en "+str(nbiter)+" étapes")
    return P

Pm = gradient(g, P0, gamma, 1)
print("valeur de la fonction objectif :", end=" ")
print(g(Pm)[0])

Réalisé en 26 étapes
valeur de la fonction objectif : 369551.54991481936


### Question 3.1

In [7]:
def line_search(g, P, initial_step = 1):
    step = initial_step
    initial_val, grad = g(P)
    val = initial_val + 1
    while(val - initial_val > 0):
        step /= 2
        val, _ = g(P - grad * step)
    return step

def line_search_gradient(g, P0, epsilon, verbose = True, maxIter = 1000):
    P = P0.copy()
    _, grad = g(P)
    nbiter = 0
    while(math.sqrt(np.sum(grad ** 2)) > epsilon and nbiter < maxIter):
        gamma = line_search(g, P)
        P_new = P - grad * gamma
        P = P_new
        _, grad = g(P)
        nbiter += 1
    if(verbose):
        print("Réalisé en "+str(nbiter)+" étapes")
    return P

Pm = line_search_gradient(g, P0, 1)
print("valeur de la fonction objectif :", end=" ")
print(g(Pm)[0])

Réalisé en 22 étapes
valeur de la fonction objectif : 369551.6667174595


### Question 3.2

Vérifier que g est quadratique est difficile car son espace de départ est un espace de matrices (et non de vecteurs). Il est donc difficile de mettre g sous la forme : 
$x \in \mathbb {R} ^{|C| \times |I|}, g(x) = ^{{\operatorname t}}\!x.A.x + ^{{\operatorname t}}\!b.x + c $ 

Mais comme $  {\nabla }^2 g(P) $ ne dépend pas de P et est définie positive, on suppose donc qu'on peut utiliser la méthode des gradients conjugués.

De plus $g$ peut se mettre sous la forme : 

$g(P) = \frac{1}{2}tr( ^{{\operatorname t}}\!P ^{{\operatorname t}}\!Q^0 Q^0 P \circ ^{{\operatorname t}}\!1_K 1_K + ^{{\operatorname t}}\!P \rho I P) + tr(^{{\operatorname t}}\!B P) + c $ 

Cette forme laisse penser qu'une fois vectoriser, on pourra mettre g sous forme quadratique

Comme on a pas mis en évidence la matrice $A$, on utilisera $tr( ^{{\operatorname t}}\!X ^{{\operatorname t}}\!Q^0 Q^0 Y \circ ^{{\operatorname t}}\!1_K 1_K + ^{{\operatorname t}}\!X \rho I Y)$ à la place de $^{{\operatorname t}}\!x.A.y$ dans l'implémentation de l'algorithme


In [8]:
def conjugate_gradient(q, x0, Q0, rho, partial):
    global mask
    tmp = mask.T.dot(mask) 
    Q0TQ0 = Q0.T.dot(Q0)
    scal = lambda v, w : v.T.dot(w).trace()
    scalA = lambda v, w : (v.T.dot(Q0TQ0).dot(w) * tmp).trace() + rho * scal(v, w)
    
    x = x0.copy()
    _, g = q(x)
    d = - g
    s = - scal(d, g) / scalA(d, d)
    x += s * d
    n = Q0.shape[1]
    
    B = x0.shape[0] * x0.shape[1]
    if(partial):
        B = 20
    
    for k in range(0, B):
        _, g = q(x)
        b = scalA(d, g) / scalA(d, d)
        d = - g + b * d
        s = - scal(d, g) / scalA(d, d)
        x += s * d
    return x

In [ ]:
# ---- ATTENTION : long ( > 10 minutes) -----
Pmconj = conjugate_gradient(g, P0, Q0, rho, False)
g(Pmconj)[0]

L'exécution complète de l'algorithme des gradients conjugués est très longue (de l'ordre de 10 minutes) car la dimension de l'espace de départ de g est très grande : 6728 ( hauteur de P $ \times $ largeur de P)

Le paramètre partial = True permet de s'arréter à la 20 itérations. 

L'éxécution complète donne un valeur à 369550.59571634443.
Cette valeur (à erreur d'arrondi numérique prêt) est supposée être le minimum de g. 
Ce que l'on vérifie en effectuant des itérations supplémentaires à partir du P trouvé : 

In [ ]:
Pm = conjugate_gradient(g, Pmconj, Q0, rho, True)
g(Pm)[0]

La valeur ne change pas du tout. On peut donc penser que l'on a bien trouvé le minimum de P (à erreur d'arrondi numérique prêt) et que g est effectivement quadratique comme on l'a supposé.

Comparée aux deux autres méthodes, cette méthode est bien plus longue. Comme l'écart relatif entre les valeurs de la fonction objectif sont faibles, on pourra se contenter d'utiliser une méthode du gradient (avec recherche linéaire ou non) ou utilisé la méthode des gradients conjugués avec un nombre restreint d'itération.
Comme ci-dessous : 

In [ ]:
Pm = conjugate_gradient(g, P0, Q0, rho, True)
g(Pm)[0]

### Question 4.1



### Question 4.2

La première ligne dans la boucle for se traduit par :
$$ \forall P, f(P_k, Q_{k-1}) \leq f(P, Q_{k-1}) $$
La deuxième ligne se traduit par :
$$ \forall Q, f(P_k, Q_k) \leq f(P_k, Q) $$

En particulier :
$$
{\begin{cases}
f(P_k, Q_{k-1}) \leq f(P_{k-1}, Q_{k-1})
\\
f(P_k, Q_k) \leq f(P_k, Q_{k-1})
\end{cases}}
$$

En combiant ces deux inégalités, on obtient que $f(P_k, Q_k) \leq f(P_{k-1}, Q_{k-1})$ et donc la suite formée par les $f(P_k, Q_k) $ est décroissante. Comme elle est aussi minorée ($f$ admet un minimum), on en conclut qu'elle converge.

### Question 4.3

Pour des raisons pratiques, on définit une fonction python objective_bis qui correspond à :
$$ h(Q) = \frac{1}{2}||1_K \circ (R - QP_0)||^2 + \frac{\rho}{2}||P_0||^2 + \frac{\rho}{2}||Q||^2 $$

In [ ]:
def objective_bis(P0, Q, R, mask, rho):
    tmp = (R - Q.dot(P0)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P0 ** 2))

    grad_Q = rho * Q - tmp.dot(P0.T)

    return val, grad_Q

def alternate_least_square(P0, Q0, epsilon, nbIter):
    global R, mask, rho
    P, Q = P0.copy(), Q0.copy()
    for k in range(nbIter):
        gP = lambda P : objective(P, Q, R, mask, rho)
        P = line_search_gradient(gP, P, epsilon, False, 30)
        gQ = lambda Q : objective_bis(P, Q, R, mask, rho)
        Q = line_search_gradient(gQ, Q, epsilon, False, 30)
    return P, Q

#prend + de 2 minutes
Pm, Qm = alternate_least_square(P0, Q0, 1, 10)
print(total_objective(Pm, Qm, R, mask, rho)[0])